# Carga/Consumo

In [230]:
import requests
import pandas as pd
from datetime import datetime, timedelta

# Get today's date
start_date = '2019-01-01'
end_date = datetime.now()

df_carga = pd.DataFrame()

# Function to obtain data from the API
def obter_carga_verificada(api_url, data_inicio, data_fim, cod_areacarga):
    url = f'{api_url}?dat_inicio={data_inicio}&dat_fim={data_fim}&cod_areacarga={cod_areacarga}'
    
    try:
        # Making the GET request to the API
        resposta = requests.get(url)
        resposta.raise_for_status()  # Raise an error for bad responses

        # Getting the content of the response in JSON format
        dados = resposta.json()
        
        return dados
    except requests.exceptions.HTTPError as errh:
        print(f"Erro HTTP: {errh}")
    except requests.exceptions.ConnectionError as errc:
        print(f"Erro de Conexão: {errc}")
    except requests.exceptions.Timeout as errt:
        print(f"Tempo de Conexão Esgotado: {errt}")
    except requests.exceptions.RequestException as err:
        print(f"Erro ao fazer a requisição: {err}")
    
    return None  # Return None if an error occurs

# Defining the API URL and query parameters
api_url = 'https://apicarga.ons.org.br/prd/cargaverificada'
  # Replace 'AREA' with the desired area code

def generate_date_ranges(start_date_str, finish_date):
    # Parse the start date string into a datetime object
    start_date = datetime.strptime(start_date_str, '%Y-%m-%d')

    # Initialize the current start date
    current_start_date = start_date

    while True:
        # Calculate the finish date for the current range
        current_finish_date = current_start_date + timedelta(days=90)  # Approximately 3 months

        # Format the finish date as a string
        end = current_finish_date.strftime('%Y-%m-%d')

        # Stop if the current finish date exceeds the given finish date
        if current_finish_date > finish_date:
            break

        # Get data from the API for the current date range
        dados_carga = obter_carga_verificada(api_url, current_start_date.strftime('%Y-%m-%d'), end, cod_areacarga)
        
        # If the data is valid, concatenate it to the DataFrame
        if dados_carga is not None:
            df_temp = pd.DataFrame(dados_carga)  # Assuming the data is in a suitable format for DataFrame
            global df_carga  # Use the global DataFrame
            df_carga = pd.concat([df_carga, df_temp], ignore_index=True)

        # Update the current start date for the next iteration
        current_start_date = current_finish_date

# Call the function with the current date as the finish date


def values_treatment (df_carga):
    df_carga = df_carga[df_carga['val_cargaglobal'] != 0]
    df_carga = df_carga.drop_duplicates()
    df_carga['din_referenciautc'] = pd.to_datetime(df_carga['din_referenciautc'])

    # Extract year, month, day, and hour for grouping
    df_carga['year'] = df_carga['din_referenciautc'].dt.year
    df_carga['month'] = df_carga['din_referenciautc'].dt.month
    df_carga['day'] = df_carga['din_referenciautc'].dt.day
    df_carga['hour'] = df_carga['din_referenciautc'].dt.hour

    # Group by year, month, day, and hour, then sum the values and keep other columns
    result = df_carga.groupby(['year', 'month', 'day', 'hour'], as_index=False).agg(
        val_cargaglobal=('val_cargaglobal', 'mean'),
        val_cargaglobalcons=('val_cargaglobalcons', 'mean'),
        val_cargaglobalsmmgd=('val_cargaglobalsmmgd', 'mean'),
        val_cargasupervisionada=('val_cargasupervisionada', 'mean'),
        val_carganaosupervisionada=('val_carganaosupervisionada', 'mean'),
        val_cargammgd=('val_cargammgd', 'mean'),
        val_consistencia=('val_consistencia', 'mean'),
        cod_areacarga=('cod_areacarga', 'first'),  # Keep the first occurrence
        din_atualizacao=('din_atualizacao', 'first'),    # Keep the first occurrence
        dat_referencia=('dat_referencia', 'first')  # Keep the first occurrence
    )

    # Create a new datetime column
    result['din_referenciautc'] = pd.to_datetime(result[['year', 'month', 'day', 'hour']].assign(minute=0))

    # Convert back to the original string format
    result['din_referenciautc'] = result['din_referenciautc'].dt.strftime('%Y-%m-%dT%H:%M:%S.000Z')

    # Drop the year, month, day, hour columns if no longer needed
    result = result.drop(columns=['year', 'month', 'day', 'hour'])

    return result
cod_areacarga = 'SECO'
generate_date_ranges(start_date, end_date)
resultSECO = values_treatment(df_carga)

df_carga = pd.DataFrame()
cod_areacarga = 'NE'
generate_date_ranges(start_date, end_date)
resultNE = values_treatment(df_carga)

df_carga = pd.DataFrame()
cod_areacarga = 'N'
generate_date_ranges(start_date, end_date)
resultN = values_treatment(df_carga)

df_carga = pd.DataFrame()
cod_areacarga = 'S'
generate_date_ranges(start_date, end_date)
resultS = values_treatment(df_carga)
# Display the resulting DataFrame

df_carga = pd.concat([resultSECO, resultNE, resultN, resultS])


new_order = ['cod_areacarga', 'din_atualizacao', 'dat_referencia', 'din_referenciautc', 'val_cargaglobal', 'val_cargaglobalcons', 'val_cargaglobalsmmgd', 'val_cargasupervisionada', 'val_carganaosupervisionada', 'val_cargammgd', 'val_consistencia']
df_carga = df_carga[new_order]
df_carga = df_carga.drop_duplicates().reset_index(drop=True)
df_carga

Erro ao fazer a requisição: Expecting value: line 9 column 33 (char 318)
Erro ao fazer a requisição: Expecting value: line 9 column 33 (char 316)
Erro ao fazer a requisição: Expecting value: line 9 column 33 (char 315)
Erro ao fazer a requisição: Expecting value: line 9 column 33 (char 313)


,cod_areacarga,din_atualizacao,dat_referencia,din_referenciautc,val_cargaglobal,val_cargaglobalcons,val_cargaglobalsmmgd,val_cargasupervisionada,val_carganaosupervisionada,val_cargammgd,val_consistencia
0,SECO,2023-04-28T08:46:08.399Z,2019-04-01,2019-04-01T03:00:00.000Z,35361.2270,35361.2270,35315.8360,35315.8360,0.00000,45.38990,0.0
1,SECO,2023-04-28T08:46:08.399Z,2019-04-01,2019-04-01T04:00:00.000Z,34080.9905,34080.9905,34035.5995,34035.5995,0.00000,45.39010,0.0
2,SECO,2023-04-28T08:46:08.399Z,2019-04-01,2019-04-01T05:00:00.000Z,32914.8085,32914.8085,32869.4195,32869.4195,0.00000,45.38975,0.0
3,SECO,2023-04-28T08:46:08.399Z,2019-04-01,2019-04-01T06:00:00.000Z,32271.3705,32271.3705,32225.9795,32225.9795,0.00000,45.39100,0.0
4,SECO,2023-04-28T08:46:08.399Z,2019-04-01,2019-04-01T07:00:00.000Z,32112.6400,32112.6400,32067.2525,32067.2525,0.00000,45.38625,0.0
...,...,...,...,...,...,...,...,...,...,...,...
190175,S,2024-09-30T03:19:28.098Z,2024-09-01,2024-09-01T23:00:00.000Z,13160.7880,13160.7880,13117.2080,12286.2405,830.96780,43.58000,0.0
190176,S,2024-09-30T03:19:28.098Z,2024-09-01,2024-09-02T00:00:00.000Z,12677.8850,12677.8850,12634.3050,11803.3875,830.91725,43.58000,0.0
190177,S,2024-09-30T03:19:28.098Z,2024-09-01,2024-09-02T01:00:00.000Z,11944.9425,11944.9425,11901.1175,11071.8195,829.29760,43.82530,0.0
190178,S,2024-09-30T03:19:28.098Z,2024-09-01,2024-09-02T02:00:00.000Z,10973.6000,10973.6000,10930.0200,10103.2540,826.76605,43.58000,0.0


In [1]:
import pandas as pd

failure = False; i = 2000
df_carga = pd.DataFrame
while failure == False:
    url = f'https://ons-aws-prod-opendata.s3.amazonaws.com/dataset/carga_energia_di/CARGA_ENERGIA_{i}.csv'
    try:
        # Lendo o CSV diretamente da URL com delimitador ';'
        dados_carga = pd.read_csv(url, delimiter=';')
    except Exception as e:
        # print(f"Erro ao carregar o arquivo CSV: {e}")
        failure = True
    if i == 2000:
        df_carga = dados_carga
    elif failure == False: 
        df_carga = pd.concat([df_carga, dados_carga])
    i = i + 1
df_carga.drop(columns= 'nom_subsistema', inplace=True)
df_carga = df_carga.reset_index(drop=True)
df_carga

,id_subsistema,din_instante,val_cargaenergiamwmed
0,N,2000-01-01,2243.512500
1,NE,2000-01-01,4646.370833
2,S,2000-01-01,4800.650000
3,SE,2000-01-01,19045.995833
4,N,2000-01-02,2259.808333
...,...,...,...
36264,SE,2024-10-26,NaN
36265,N,2024-10-27,NaN
36266,NE,2024-10-27,NaN
36267,S,2024-10-27,NaN


In [237]:
df_carga[df_carga['val_cargaenergiamwmed'].isna()]

,id_subsistema,din_instante,val_cargaenergiamwmed
19128,S,2013-02-02 00:00:01,NaN
20333,N,2013-12-01 00:00:00,NaN
20334,NE,2013-12-01 00:00:00,NaN
20335,S,2013-12-01 00:00:00,NaN
20336,SE,2013-12-01 00:00:00,NaN
20581,N,2014-02-01,NaN
20582,NE,2014-02-01,NaN
20583,S,2014-02-01,NaN
20584,SE,2014-02-01,NaN
22309,N,2015-04-09,NaN


In [2]:
df_carga.to_csv('Carga_Consumo_atualizado.csv')

# ENA

In [ ]:
# import pandas as pd

# # URL do arquivo CSV
# url = 'https://ons-aws-prod-opendata.s3.amazonaws.com/dataset/ena_subsistema_di/ENA_DIARIO_SUBSISTEMA_2024.csv'

# def carregar_dados_csv(url):
#     try:
#         # Lendo o CSV diretamente da URL com delimitador ';'
#         df = pd.read_csv(url, delimiter=';')
#         return df
#     except Exception as e:
#         print(f"Erro ao carregar o arquivo CSV: {e}")

# # Carregando os dados do CSV
# dados = carregar_dados_csv(url)
# dados
import pandas as pd

failure = False; i = 2000
df_earm = pd.DataFrame
while failure == False:
    url = f'https://ons-aws-prod-opendata.s3.amazonaws.com/dataset/ena_subsistema_di/ENA_DIARIO_SUBSISTEMA_{i}.csv'
    try:
        # Lendo o CSV diretamente da URL com delimitador ';'
        dados_ena = pd.read_csv(url, delimiter=';')
    except Exception as e:
        # print(f"Erro ao carregar o arquivo CSV: {e}")
        failure = True
    if i == 2000:
        df_ena = dados_ena
    elif failure == False: 
        df_ena = pd.concat([df_ena, dados_ena])
    i = i + 1
df_ena.replace({'NORDESTE': 'NE', 'NORTE': 'N','SUL': 'S','SUDESTE': 'SE'}, inplace=True)
df_ena = df_ena.drop(columns=["ena_bruta_regiao_percentualmlt", "ena_armazenavel_regiao_percentualmlt"])
df_ena['ena_data'] = pd.to_datetime(df_ena['ena_data'])
df_ena['month'] = df_ena['ena_data'].dt.month
def month_value_condition(row):
    month = row['month']
    sub = row['nom_subsistema']
    ena_bruta = row['ena_bruta_regiao_mwmed']
    if sub == 'SE':
        switcher = {
            1: (ena_bruta*0.1) / 65.801,
            2: (ena_bruta*0.1) / 70.894,
            3: (ena_bruta*0.1) / 69.077,
            4: (ena_bruta*0.1) / 54.673,
            5: (ena_bruta*0.1) / 39.825,
            6: (ena_bruta*0.1) / 32.509,
            7: (ena_bruta*0.1) / 25.574,
            8: (ena_bruta*0.1) / 20.521,
            9: (ena_bruta*0.1) / 19.627,
            10: (ena_bruta*0.1) / 23.653,
            11: (ena_bruta*0.1) / 31.416,
            12: (ena_bruta*0.1) / 48.056
        }
        return switcher.get(month, "Invalid month")
    elif sub == 'N':
        switcher = {
            1: (ena_bruta*0.1) / 15.832,
            2: (ena_bruta*0.1) / 22.811,
            3: (ena_bruta*0.1) / 26.961,
            4: (ena_bruta*0.1) / 27.160,
            5: (ena_bruta*0.1) / 20.486,
            6: (ena_bruta*0.1) / 10.695,
            7: (ena_bruta*0.1) / 5.253,
            8: (ena_bruta*0.1) / 3.184,
            9: (ena_bruta*0.1) / 2.267,
            10: (ena_bruta*0.1) / 2.383,
            11: (ena_bruta*0.1) / 4.063,
            12: (ena_bruta*0.1) / 8.415
        }
        return switcher.get(month, "Invalid month")
    elif sub == 'NE':
        switcher = {
            1: (ena_bruta*0.1) / 13.479,
            2: (ena_bruta*0.1) / 14.238,
            3: (ena_bruta*0.1) / 14.148,
            4: (ena_bruta*0.1) / 11.448,
            5: (ena_bruta*0.1) / 6.905,
            6: (ena_bruta*0.1) / 4.559,
            7: (ena_bruta*0.1) / 3.761,
            8: (ena_bruta*0.1) / 3.279,
            9: (ena_bruta*0.1) / 2.933,
            10: (ena_bruta*0.1) / 3.189,
            11: (ena_bruta*0.1) / 5.279,
            12: (ena_bruta*0.1) / 9.834
        }
        return switcher.get(month, "Invalid month")
    elif sub == 'S':
        switcher = {
            1: (ena_bruta*0.1) / 7.548,
            2: (ena_bruta*0.1) / 8.272,
            3: (ena_bruta*0.1) / 7.066,
            4: (ena_bruta*0.1) / 6.545,
            5: (ena_bruta*0.1) / 8.630,
            6: (ena_bruta*0.1) / 10.598,
            7: (ena_bruta*0.1) / 10.900,
            8: (ena_bruta*0.1) / 10.004,
            9: (ena_bruta*0.1) / 11.628,
            10: (ena_bruta*0.1) / 13.360,
            11: (ena_bruta*0.1) / 9.311,
            12: (ena_bruta*0.1) / 7.415
        }
        return switcher.get(month, "Invalid month")
df_ena['ena_bruta_regiao_percentualmlt'] = df_ena.apply(month_value_condition, axis=1)
df_ena.drop(columns= ['month']).reset_index(drop=True)

ESSE É O CERTO

In [6]:
import pandas as pd

#TEM QUE SER *ESSE* ARQUIVO
historico = pd.read_excel("Enas_Subsistemas_1931-2022.xlsx")

# Função para calcular médias, mínimos e máximos, e atualizar porcentagens
def calcular_estatisticas_e_atualizar_porcentagens(df, novo_ano, novos_valores):

    if novo_ano in df['Ano'].values:
        print(f"O ano {novo_ano} já existe no DataFrame. Nenhuma ação será realizada.")
        return df
    df = df.drop(columns=['Subsistema', '(min)MWmed',	'(min)% med.',	'(max)MWmed', '(max)% med.','(jan)% med.', '(fev)% med.', '(mar)% med.', '(abr)% med.', '(mai)% med.','(jun)% med.', '(jul)% med.', '(ago)% med.', '(set)% med.', '(out)% med.', '(nov)% med.', '(dez)% med.',])
    try : df =  df.drop(columns= ['(jandez)MWmed','(jandez)% med.'])
    except : df =  df.drop(columns= ['(jnd)MWmed','(jnd)% med.'])
    df = df[:-1]
    nova_linha = {'Ano': novo_ano}
    for mes, valor in zip(['(jan)MWmed', '(fev)MWmed', '(mar)MWmed', '(abr)MWmed', '(mai)MWmed', '(jun)MWmed', '(jul)MWmed', '(ago)MWmed', '(set)MWmed', '(out)MWmed', '(nov)MWmed', '(dez)MWmed'], novos_valores):
        nova_linha[mes] = valor
        
    df = pd.concat([df, pd.DataFrame([nova_linha])], ignore_index=True)
        # Drop the last row (old mean) if it exists
    df['(min)MWmed'] = df.iloc[:, 1:13].min(axis=1)
    df['(jnd)MWmed'] = df.iloc[:, 1:13].mean(axis=1)
    df['(max)MWmed'] = df.iloc[:, 1:13].max(axis=1)
    meses = ['(jan)MWmed', '(fev)MWmed', '(mar)MWmed', '(abr)MWmed', '(mai)MWmed', '(jun)MWmed', '(jul)MWmed', '(ago)MWmed', '(set)MWmed', '(out)MWmed', '(nov)MWmed', '(dez)MWmed']
    mjdm = ['(min)MWmed', '(jnd)MWmed', '(max)MWmed']
    ano_atual = "media"  # Próximo ano

    # Calcular médias mensais
    media_row = {'Ano': ano_atual}
    for mes in meses:
        media_row[mes] = df[mes].mean()  # Média dos valores para cada mês
        
    means = [df[mes].mean() for mes in meses]

    for idx in range(len(df)):
        for i in range(len(meses)):
            value_col = df.iloc[idx][meses[i]]  # Valor atual
            if pd.notna(value_col) and means[i] != 0:  # Previne divisão por zero
                df.loc[idx, f'{meses[i][0:5]}% med.'] = (value_col / means[i]) * 100
            else:
                df.loc[idx, f'{meses[i][0:5]}% med.'] = 100  # Define como None se valor for NaN ou média for 0

    means2 = df['(jnd)MWmed'].mean()

    for idx in range(len(df)):
        for i in range(len(mjdm)):
            value_col = df.iloc[idx][mjdm[i]]  # Valor atual
            if pd.notna(value_col) and means2 != 0:  # Previne divisão por zero
                df.loc[idx, f'{mjdm[i][0:5]}% med.'] = (value_col / means2) * 100
            else:
                df.loc[idx, f'{mjdm[i][0:5]}% med.'] = 100 
    
    # Adiciona a nova linha com as estatísticas
    df = pd.concat([df, pd.DataFrame([media_row])], ignore_index=True)
    df = df.fillna(100)    

    return df

def inicio_SE(historicoSE):

    # Escreva os valores para a região SE na ordem dos meses (jan, fev, mar, ..., dez)
    values_input = [71658, 78130, 52904,  41024, 26938, 24249, 16822, 16036, 16085, 24508, 24449, 43079]

    historicoSE = historicoSE[historicoSE['Ano'] != 2022]

    # Calcula estatísticas e atualiza porcentagens
    historicoSE = calcular_estatisticas_e_atualizar_porcentagens(historicoSE, 2022, values_input)
    historicoSE['Subsistema'] = "SE"
    return historicoSE

def inicio_N(historicoN):

    # Escreva os valores para a região N na ordem dos meses (jan, fev, mar, ..., dez)
    values_input = [35162, 30798, 32737,  29655, 17111, 8829, 4573, 2739, 1763, 1718, 3841, 10253]

    historicoN = historicoN[historicoN['Ano'] != 2022]


    # Calcula estatísticas e atualiza porcentagens
    historicoN = calcular_estatisticas_e_atualizar_porcentagens(historicoN, 2022, values_input)
    historicoN['Subsistema'] = "N"
    return historicoN

def inicio_S(historicoS):

    # Escreva os valores para a região S na ordem dos meses (jan, fev, mar, ..., dez)
    values_input = [2802, 2751, 6990,  10176, 18906, 24482, 7823, 12663, 9727, 20738, 7820, 8163]

    historicoS = historicoS[historicoS['Ano'] != 2022]

    # Calcula estatísticas e atualiza porcentagens
    historicoS = calcular_estatisticas_e_atualizar_porcentagens(historicoS, 2022, values_input)
    historicoS['Subsistema'] = "S"
    return historicoS

def inicio_NE(historicoNE):

    # Escreva os valores para a região NE na ordem dos meses (jan, fev, mar, ..., dez)
    values_input = [17924, 20879, 17281,  7449, 3678, 2857, 2445, 2056, 2234, 1912, 4270, 9539]

    historicoNE = historicoNE[historicoNE['Ano'] != 2022]

    # Calcula estatísticas e atualiza porcentagens
    historicoNE = calcular_estatisticas_e_atualizar_porcentagens(historicoNE, 2022, values_input)
    historicoNE['Subsistema'] = "NE"
    return historicoNE

# Categorização e inicialização do histórico do subsistema SE
historicoSE = historico[historico['Subsistema'] == 'SECO']
historicoSE = inicio_SE(historicoSE)

# Escreva os valores para a região SE na ordem dos meses (jan, fev, mar, ..., dez)
values_inputSE = [71658, 78130, 52904,  41024, 26938, 24249, 16822, 16036, 16085, 24508, 24449, 43079]

# Escreva o ano que você deseja adicionar
anoSE = 2022

# Calcula estatísticas e atualiza porcentagens
historicoSE = calcular_estatisticas_e_atualizar_porcentagens(historicoSE, anoSE, values_inputSE)
historicoSE['Subsistema'] = "SE"

# Categorização do histórico do subsistema N
historicoN = historico[historico['Subsistema'] == 'N']
historicoN = inicio_N(historicoN)

 # Escreva os valores para a região N na ordem dos meses (jan, fev, mar, ..., dez)
values_input = [35162, 30798, 32737,  29655, 17111, 8829, 4573, 2739, 1763, 1718, 3841, 10253]

# Escreva o ano que você deseja adicionar
ano = 2022

# Calcula estatísticas e atualiza porcentagens
historicoN = calcular_estatisticas_e_atualizar_porcentagens(historicoN, ano, values_input)
historicoN['Subsistema'] = "N"


# Categorização do histórico do subsistema S
historicoS = historico[historico['Subsistema'] == 'S']
historicoS = inicio_S(historicoS)

# Escreva os valores para a região S na ordem dos meses (jan, fev, mar, ..., dez)
values_inputS = [2802, 2751, 6990,  10176, 18906, 24482, 7823, 12663, 9727, 20738, 7820, 8163]

# Escreva o ano que você deseja adicionar
anoS = 2022
# Calcula estatísticas e atualiza porcentagens

historicoS = calcular_estatisticas_e_atualizar_porcentagens(historicoS, anoS, values_inputS)
historicoS['Subsistema'] = "S"

# Categorização do histórico do subsistema NE
historicoNE = historico[historico['Subsistema'] == 'NE']
historicoNE = inicio_NE(historicoNE)

 # Escreva os valores para a região NE na ordem dos meses (jan, fev, mar, ..., dez)
values_inputNE = [17924, 20879, 17281,  7449, 3678, 2857, 2445, 2056, 2234, 1912, 4270, 9539]

# Escreva o ano que você deseja adicionar
anoNE = 2022

# Calcula estatísticas e atualiza porcentagens
historicoNE = calcular_estatisticas_e_atualizar_porcentagens(historicoNE, anoNE, values_inputNE)
historicoNE['Subsistema'] = "NE"

df_mlt = pd.concat([historicoSE, historicoS, historicoN, historicoNE]).reset_index(drop=True)

failure = False; i = 2000
df_earm = pd.DataFrame
while failure == False:
    url = f'https://ons-aws-prod-opendata.s3.amazonaws.com/dataset/ena_subsistema_di/ENA_DIARIO_SUBSISTEMA_{i}.csv'
    try:
        # Lendo o CSV diretamente da URL com delimitador ';'
        dados_ena = pd.read_csv(url, delimiter=';')
    except Exception as e:
        # print(f"Erro ao carregar o arquivo CSV: {e}")
        failure = True
    if i == 2000:
        df_ena = dados_ena
    elif failure == False: 
        df_ena = pd.concat([df_ena, dados_ena])
    i = i + 1
df_ena.replace({'NORDESTE': 'NE', 'NORTE': 'N','SUL': 'S','SUDESTE': 'SE'}, inplace=True)
df_ena = df_ena.drop(columns=["ena_bruta_regiao_percentualmlt", "ena_armazenavel_regiao_percentualmlt"])
df_ena['ena_data'] = pd.to_datetime(df_ena['ena_data'])
df_ena['month'] = df_ena['ena_data'].dt.month
def month_value_condition_bruta(row, df):
    month = row['month']
    sub = row['nom_subsistema']
    ena_bruta = row['ena_bruta_regiao_mwmed']
    if sub == 'SE':
        switcher = {
            1: (ena_bruta*100) / df.loc[(df['Subsistema'] == 'SE') & (df['Ano'] == 'media'), '(jan)MWmed'].values[0],
            2: (ena_bruta*100) / df.loc[(df['Subsistema'] == 'SE') & (df['Ano'] == 'media'), '(fev)MWmed'].values[0],
            3: (ena_bruta*100) / df.loc[(df['Subsistema'] == 'SE') & (df['Ano'] == 'media'), '(mar)MWmed'].values[0],
            4: (ena_bruta*100) / df.loc[(df['Subsistema'] == 'SE') & (df['Ano'] == 'media'), '(abr)MWmed'].values[0],
            5: (ena_bruta*100) / df.loc[(df['Subsistema'] == 'SE') & (df['Ano'] == 'media'), '(mai)MWmed'].values[0],
            6: (ena_bruta*100) / df.loc[(df['Subsistema'] == 'SE') & (df['Ano'] == 'media'), '(jun)MWmed'].values[0],
            7: (ena_bruta*100) / df.loc[(df['Subsistema'] == 'SE') & (df['Ano'] == 'media'), '(jul)MWmed'].values[0],
            8: (ena_bruta*100) / df.loc[(df['Subsistema'] == 'SE') & (df['Ano'] == 'media'), '(ago)MWmed'].values[0],
            9: (ena_bruta*100) / df.loc[(df['Subsistema'] == 'SE') & (df['Ano'] == 'media'), '(set)MWmed'].values[0],
            10: (ena_bruta*100) / df.loc[(df['Subsistema'] == 'SE') & (df['Ano'] == 'media'), '(out)MWmed'].values[0],
            11: (ena_bruta*100) / df.loc[(df['Subsistema'] == 'SE') & (df['Ano'] == 'media'), '(nov)MWmed'].values[0],
            12: (ena_bruta*100) / df.loc[(df['Subsistema'] == 'SE') & (df['Ano'] == 'media'), '(dez)MWmed'].values[0]
        }
        return switcher.get(month, "Invalid month")
    elif sub == 'N':
        switcher = {
            1: (ena_bruta*100) / df.loc[(df['Subsistema'] == 'N') & (df['Ano'] == 'media'), '(jan)MWmed'].values[0],
            2: (ena_bruta*100) / df.loc[(df['Subsistema'] == 'N') & (df['Ano'] == 'media'), '(fev)MWmed'].values[0],
            3: (ena_bruta*100) / df.loc[(df['Subsistema'] == 'N') & (df['Ano'] == 'media'), '(mar)MWmed'].values[0],
            4: (ena_bruta*100) / df.loc[(df['Subsistema'] == 'N') & (df['Ano'] == 'media'), '(abr)MWmed'].values[0],
            5: (ena_bruta*100) / df.loc[(df['Subsistema'] == 'N') & (df['Ano'] == 'media'), '(mai)MWmed'].values[0],
            6: (ena_bruta*100) / df.loc[(df['Subsistema'] == 'N') & (df['Ano'] == 'media'), '(jun)MWmed'].values[0],
            7: (ena_bruta*100) / df.loc[(df['Subsistema'] == 'N') & (df['Ano'] == 'media'), '(jul)MWmed'].values[0],
            8: (ena_bruta*100) / df.loc[(df['Subsistema'] == 'N') & (df['Ano'] == 'media'), '(ago)MWmed'].values[0],
            9: (ena_bruta*100) / df.loc[(df['Subsistema'] == 'N') & (df['Ano'] == 'media'), '(set)MWmed'].values[0],
            10: (ena_bruta*100) / df.loc[(df['Subsistema'] == 'N') & (df['Ano'] == 'media'), '(out)MWmed'].values[0],
            11: (ena_bruta*100) / df.loc[(df['Subsistema'] == 'N') & (df['Ano'] == 'media'), '(nov)MWmed'].values[0],
            12: (ena_bruta*100) / df.loc[(df['Subsistema'] == 'N') & (df['Ano'] == 'media'), '(dez)MWmed'].values[0]
        }
        return switcher.get(month, "Invalid month")
    elif sub == 'NE':
        switcher = {
            1: (ena_bruta*100) / df.loc[(df['Subsistema'] == 'NE') & (df['Ano'] == 'media'), '(jan)MWmed'].values[0],
            2: (ena_bruta*100) / df.loc[(df['Subsistema'] == 'NE') & (df['Ano'] == 'media'), '(fev)MWmed'].values[0],
            3: (ena_bruta*100) / df.loc[(df['Subsistema'] == 'NE') & (df['Ano'] == 'media'), '(mar)MWmed'].values[0],
            4: (ena_bruta*100) / df.loc[(df['Subsistema'] == 'NE') & (df['Ano'] == 'media'), '(abr)MWmed'].values[0],
            5: (ena_bruta*100) / df.loc[(df['Subsistema'] == 'NE') & (df['Ano'] == 'media'), '(mai)MWmed'].values[0],
            6: (ena_bruta*100) / df.loc[(df['Subsistema'] == 'NE') & (df['Ano'] == 'media'), '(jun)MWmed'].values[0],
            7: (ena_bruta*100) / df.loc[(df['Subsistema'] == 'NE') & (df['Ano'] == 'media'), '(jul)MWmed'].values[0],
            8: (ena_bruta*100) / df.loc[(df['Subsistema'] == 'NE') & (df['Ano'] == 'media'), '(ago)MWmed'].values[0],
            9: (ena_bruta*100) / df.loc[(df['Subsistema'] == 'NE') & (df['Ano'] == 'media'), '(set)MWmed'].values[0],
            10: (ena_bruta*100) / df.loc[(df['Subsistema'] == 'NE') & (df['Ano'] == 'media'), '(out)MWmed'].values[0],
            11: (ena_bruta*100) / df.loc[(df['Subsistema'] == 'NE') & (df['Ano'] == 'media'), '(nov)MWmed'].values[0],
            12: (ena_bruta*100) / df.loc[(df['Subsistema'] == 'NE') & (df['Ano'] == 'media'), '(dez)MWmed'].values[0]
        }
        return switcher.get(month, "Invalid month")
    elif sub == 'S':
        switcher = {
            1: (ena_bruta*100) / df.loc[(df['Subsistema'] == 'S') & (df['Ano'] == 'media'), '(jan)MWmed'].values[0],
            2: (ena_bruta*100) / df.loc[(df['Subsistema'] == 'S') & (df['Ano'] == 'media'), '(fev)MWmed'].values[0],
            3: (ena_bruta*100) / df.loc[(df['Subsistema'] == 'S') & (df['Ano'] == 'media'), '(mar)MWmed'].values[0],
            4: (ena_bruta*100) / df.loc[(df['Subsistema'] == 'S') & (df['Ano'] == 'media'), '(abr)MWmed'].values[0],
            5: (ena_bruta*100) / df.loc[(df['Subsistema'] == 'S') & (df['Ano'] == 'media'), '(mai)MWmed'].values[0],
            6: (ena_bruta*100) / df.loc[(df['Subsistema'] == 'S') & (df['Ano'] == 'media'), '(jun)MWmed'].values[0],
            7: (ena_bruta*100) / df.loc[(df['Subsistema'] == 'S') & (df['Ano'] == 'media'), '(jul)MWmed'].values[0],
            8: (ena_bruta*100) / df.loc[(df['Subsistema'] == 'S') & (df['Ano'] == 'media'), '(ago)MWmed'].values[0],
            9: (ena_bruta*100) / df.loc[(df['Subsistema'] == 'S') & (df['Ano'] == 'media'), '(set)MWmed'].values[0],
            10: (ena_bruta*100) / df.loc[(df['Subsistema'] == 'S') & (df['Ano'] == 'media'), '(out)MWmed'].values[0],
            11: (ena_bruta*100) / df.loc[(df['Subsistema'] == 'S') & (df['Ano'] == 'media'), '(nov)MWmed'].values[0],
            12: (ena_bruta*100) / df.loc[(df['Subsistema'] == 'S') & (df['Ano'] == 'media'), '(dez)MWmed'].values[0]
        }
        return switcher.get(month, "Invalid month")
def month_value_condition_armazenavel(row, df):
    month = row['month']
    sub = row['nom_subsistema']
    ena_armazenavel = row['ena_armazenavel_regiao_mwmed']
    if sub == 'SE':
        switcher = {
            1: (ena_armazenavel*100) / df.loc[(df['Subsistema'] == 'SE') & (df['Ano'] == 'media'), '(jan)MWmed'].values[0],
            2: (ena_armazenavel*100) / df.loc[(df['Subsistema'] == 'SE') & (df['Ano'] == 'media'), '(fev)MWmed'].values[0],
            3: (ena_armazenavel*100) / df.loc[(df['Subsistema'] == 'SE') & (df['Ano'] == 'media'), '(mar)MWmed'].values[0],
            4: (ena_armazenavel*100) / df.loc[(df['Subsistema'] == 'SE') & (df['Ano'] == 'media'), '(abr)MWmed'].values[0],
            5: (ena_armazenavel*100) / df.loc[(df['Subsistema'] == 'SE') & (df['Ano'] == 'media'), '(mai)MWmed'].values[0],
            6: (ena_armazenavel*100) / df.loc[(df['Subsistema'] == 'SE') & (df['Ano'] == 'media'), '(jun)MWmed'].values[0],
            7: (ena_armazenavel*100) / df.loc[(df['Subsistema'] == 'SE') & (df['Ano'] == 'media'), '(jul)MWmed'].values[0],
            8: (ena_armazenavel*100) / df.loc[(df['Subsistema'] == 'SE') & (df['Ano'] == 'media'), '(ago)MWmed'].values[0],
            9: (ena_armazenavel*100) / df.loc[(df['Subsistema'] == 'SE') & (df['Ano'] == 'media'), '(set)MWmed'].values[0],
            10: (ena_armazenavel*100) / df.loc[(df['Subsistema'] == 'SE') & (df['Ano'] == 'media'), '(out)MWmed'].values[0],
            11: (ena_armazenavel*100) / df.loc[(df['Subsistema'] == 'SE') & (df['Ano'] == 'media'), '(nov)MWmed'].values[0],
            12: (ena_armazenavel*100) / df.loc[(df['Subsistema'] == 'SE') & (df['Ano'] == 'media'), '(dez)MWmed'].values[0]
        }
        return switcher.get(month, "Invalid month")
    elif sub == 'N':
        switcher = {
            1: (ena_armazenavel*100) / df.loc[(df['Subsistema'] == 'N') & (df['Ano'] == 'media'), '(jan)MWmed'].values[0],
            2: (ena_armazenavel*100) / df.loc[(df['Subsistema'] == 'N') & (df['Ano'] == 'media'), '(fev)MWmed'].values[0],
            3: (ena_armazenavel*100) / df.loc[(df['Subsistema'] == 'N') & (df['Ano'] == 'media'), '(mar)MWmed'].values[0],
            4: (ena_armazenavel*100) / df.loc[(df['Subsistema'] == 'N') & (df['Ano'] == 'media'), '(abr)MWmed'].values[0],
            5: (ena_armazenavel*100) / df.loc[(df['Subsistema'] == 'N') & (df['Ano'] == 'media'), '(mai)MWmed'].values[0],
            6: (ena_armazenavel*100) / df.loc[(df['Subsistema'] == 'N') & (df['Ano'] == 'media'), '(jun)MWmed'].values[0],
            7: (ena_armazenavel*100) / df.loc[(df['Subsistema'] == 'N') & (df['Ano'] == 'media'), '(jul)MWmed'].values[0],
            8: (ena_armazenavel*100) / df.loc[(df['Subsistema'] == 'N') & (df['Ano'] == 'media'), '(ago)MWmed'].values[0],
            9: (ena_armazenavel*100) / df.loc[(df['Subsistema'] == 'N') & (df['Ano'] == 'media'), '(set)MWmed'].values[0],
            10: (ena_armazenavel*100) / df.loc[(df['Subsistema'] == 'N') & (df['Ano'] == 'media'), '(out)MWmed'].values[0],
            11: (ena_armazenavel*100) / df.loc[(df['Subsistema'] == 'N') & (df['Ano'] == 'media'), '(nov)MWmed'].values[0],
            12: (ena_armazenavel*100) / df.loc[(df['Subsistema'] == 'N') & (df['Ano'] == 'media'), '(dez)MWmed'].values[0]
        }
        return switcher.get(month, "Invalid month")
    elif sub == 'NE':
        switcher = {
            1: (ena_armazenavel*100) / df.loc[(df['Subsistema'] == 'NE') & (df['Ano'] == 'media'), '(jan)MWmed'].values[0],
            2: (ena_armazenavel*100) / df.loc[(df['Subsistema'] == 'NE') & (df['Ano'] == 'media'), '(fev)MWmed'].values[0],
            3: (ena_armazenavel*100) / df.loc[(df['Subsistema'] == 'NE') & (df['Ano'] == 'media'), '(mar)MWmed'].values[0],
            4: (ena_armazenavel*100) / df.loc[(df['Subsistema'] == 'NE') & (df['Ano'] == 'media'), '(abr)MWmed'].values[0],
            5: (ena_armazenavel*100) / df.loc[(df['Subsistema'] == 'NE') & (df['Ano'] == 'media'), '(mai)MWmed'].values[0],
            6: (ena_armazenavel*100) / df.loc[(df['Subsistema'] == 'NE') & (df['Ano'] == 'media'), '(jun)MWmed'].values[0],
            7: (ena_armazenavel*100) / df.loc[(df['Subsistema'] == 'NE') & (df['Ano'] == 'media'), '(jul)MWmed'].values[0],
            8: (ena_armazenavel*100) / df.loc[(df['Subsistema'] == 'NE') & (df['Ano'] == 'media'), '(ago)MWmed'].values[0],
            9: (ena_armazenavel*100) / df.loc[(df['Subsistema'] == 'NE') & (df['Ano'] == 'media'), '(set)MWmed'].values[0],
            10: (ena_armazenavel*100) / df.loc[(df['Subsistema'] == 'NE') & (df['Ano'] == 'media'), '(out)MWmed'].values[0],
            11: (ena_armazenavel*100) / df.loc[(df['Subsistema'] == 'NE') & (df['Ano'] == 'media'), '(nov)MWmed'].values[0],
            12: (ena_armazenavel*100) / df.loc[(df['Subsistema'] == 'NE') & (df['Ano'] == 'media'), '(dez)MWmed'].values[0]
        }
        return switcher.get(month, "Invalid month")
    elif sub == 'S':
        switcher = {
            1: (ena_armazenavel*100) / df.loc[(df['Subsistema'] == 'S') & (df['Ano'] == 'media'), '(jan)MWmed'].values[0],
            2: (ena_armazenavel*100) / df.loc[(df['Subsistema'] == 'S') & (df['Ano'] == 'media'), '(fev)MWmed'].values[0],
            3: (ena_armazenavel*100) / df.loc[(df['Subsistema'] == 'S') & (df['Ano'] == 'media'), '(mar)MWmed'].values[0],
            4: (ena_armazenavel*100) / df.loc[(df['Subsistema'] == 'S') & (df['Ano'] == 'media'), '(abr)MWmed'].values[0],
            5: (ena_armazenavel*100) / df.loc[(df['Subsistema'] == 'S') & (df['Ano'] == 'media'), '(mai)MWmed'].values[0],
            6: (ena_armazenavel*100) / df.loc[(df['Subsistema'] == 'S') & (df['Ano'] == 'media'), '(jun)MWmed'].values[0],
            7: (ena_armazenavel*100) / df.loc[(df['Subsistema'] == 'S') & (df['Ano'] == 'media'), '(jul)MWmed'].values[0],
            8: (ena_armazenavel*100) / df.loc[(df['Subsistema'] == 'S') & (df['Ano'] == 'media'), '(ago)MWmed'].values[0],
            9: (ena_armazenavel*100) / df.loc[(df['Subsistema'] == 'S') & (df['Ano'] == 'media'), '(set)MWmed'].values[0],
            10: (ena_armazenavel*100) / df.loc[(df['Subsistema'] == 'S') & (df['Ano'] == 'media'), '(out)MWmed'].values[0],
            11: (ena_armazenavel*100) / df.loc[(df['Subsistema'] == 'S') & (df['Ano'] == 'media'), '(nov)MWmed'].values[0],
            12: (ena_armazenavel*100) / df.loc[(df['Subsistema'] == 'S') & (df['Ano'] == 'media'), '(dez)MWmed'].values[0]
        }
        return switcher.get(month, "Invalid month")
df_ena['ena_bruta_regiao_percentualmlt'] = df_ena.apply(lambda row: month_value_condition_bruta(row, df_mlt), axis=1)
df_ena['ena_armazenavel_regiao_percentualmlt'] = df_ena.apply(lambda row: month_value_condition_armazenavel(row, df_mlt), axis=1)
df_ena.drop(columns= ['month', 'nom_subsistema']).reset_index(drop=True)


O ano 2022 já existe no DataFrame. Nenhuma ação será realizada.
O ano 2022 já existe no DataFrame. Nenhuma ação será realizada.
O ano 2022 já existe no DataFrame. Nenhuma ação será realizada.
O ano 2022 já existe no DataFrame. Nenhuma ação será realizada.


,id_subsistema,ena_data,ena_bruta_regiao_mwmed,ena_armazenavel_regiao_mwmed,ena_bruta_regiao_percentualmlt,ena_armazenavel_regiao_percentualmlt
0,N,2000-01-01,10392.561523,10381.552734,65.642536,65.573001
1,NE,2000-01-01,10174.402344,9990.510742,75.481418,74.117171
2,S,2000-01-01,1772.415039,1772.415039,23.482726,23.482726
3,SE,2000-01-01,27179.791016,27097.957031,41.306259,41.181892
4,N,2000-01-02,10888.611328,10867.503906,68.775736,68.642415
...,...,...,...,...,...,...
36299,SE,2024-11-04,24651.681641,24069.228516,78.467352,76.613379
36300,N,2024-11-05,1342.713013,1331.827026,33.047675,32.779742
36301,NE,2024-11-05,2542.183105,2534.318115,48.154288,48.005309
36302,S,2024-11-05,7110.584961,6869.084961,76.364853,73.771239


In [7]:
import pandas as pd
df_ena.to_csv('Ena_atualizado.csv')

lembrar de criar um to.csv do dfmlt (de modo que atualiza o arquivo antigo) e ena
#criar um novo excel com a formatação correta e ai nao precisa inicializar cada subsistema, pq ja vai estar formatado do jeito certo, é só extrair com o modelo de até 2022, mas vamos fazer isso na etapa de validação, quando o objetivo for melhorar o tempo de execução, aí eu uso os threadpool executor
talves o PLD da pra extrair ja com a formatação tbm, que arruma as colunas. Só vamos confiramr se vai ser um historico REALMENTE ESTATICO

In [169]:
import pandas as pd
historico = pd.read_excel("Enas_Subsistemas_1931-2022.xlsx")

# Função para calcular médias, mínimos e máximos, e atualizar porcentagens
def calcular_estatisticas_e_atualizar_porcentagens(df, novo_ano, novos_valores):

    if novo_ano in df['Ano'].values:
        print(f"O ano {novo_ano} já existe no DataFrame. Nenhuma ação será realizada.")
        return df
    df = df.drop(columns=['Subsistema', '(min)MWmed',	'(min)% med.',	'(max)MWmed', '(max)% med.','(jan)% med.', '(fev)% med.', '(mar)% med.', '(abr)% med.', '(mai)% med.','(jun)% med.', '(jul)% med.', '(ago)% med.', '(set)% med.', '(out)% med.', '(nov)% med.', '(dez)% med.',])
    try : df =  df.drop(columns= ['(jandez)MWmed','(jandez)% med.'])
    except : df =  df.drop(columns= ['(jnd)MWmed','(jnd)% med.'])
    df = df[:-1]
    nova_linha = {'Ano': novo_ano}
    for mes, valor in zip(['(jan)MWmed', '(fev)MWmed', '(mar)MWmed', '(abr)MWmed', '(mai)MWmed', '(jun)MWmed', '(jul)MWmed', '(ago)MWmed', '(set)MWmed', '(out)MWmed', '(nov)MWmed', '(dez)MWmed'], novos_valores):
        nova_linha[mes] = valor
        
    df = pd.concat([df, pd.DataFrame([nova_linha])], ignore_index=True)
        # Drop the last row (old mean) if it exists
    df['(min)MWmed'] = df.iloc[:, 1:13].min(axis=1)
    df['(jnd)MWmed'] = df.iloc[:, 1:13].mean(axis=1)
    df['(max)MWmed'] = df.iloc[:, 1:13].max(axis=1)
    meses = ['(jan)MWmed', '(fev)MWmed', '(mar)MWmed', '(abr)MWmed', '(mai)MWmed', '(jun)MWmed', '(jul)MWmed', '(ago)MWmed', '(set)MWmed', '(out)MWmed', '(nov)MWmed', '(dez)MWmed']
    mjdm = ['(min)MWmed', '(jnd)MWmed', '(max)MWmed']
    ano_atual = "media"  # Próximo ano

    # Calcular médias mensais
    media_row = {'Ano': ano_atual}
    for mes in meses:
        media_row[mes] = df[mes].mean()  # Média dos valores para cada mês
        
    means = [df[mes].mean() for mes in meses]

    for idx in range(len(df)):
        for i in range(len(meses)):
            value_col = df.iloc[idx][meses[i]]  # Valor atual
            if pd.notna(value_col) and means[i] != 0:  # Previne divisão por zero
                df.loc[idx, f'{meses[i][0:5]}% med.'] = (value_col / means[i]) * 100
            else:
                df.loc[idx, f'{meses[i][0:5]}% med.'] = 100  # Define como None se valor for NaN ou média for 0

    means2 = df['(jnd)MWmed'].mean()

    for idx in range(len(df)):
        for i in range(len(mjdm)):
            value_col = df.iloc[idx][mjdm[i]]  # Valor atual
            if pd.notna(value_col) and means2 != 0:  # Previne divisão por zero
                df.loc[idx, f'{mjdm[i][0:5]}% med.'] = (value_col / means2) * 100
            else:
                df.loc[idx, f'{mjdm[i][0:5]}% med.'] = 100 
    
    # Adiciona a nova linha com as estatísticas
    df = pd.concat([df, pd.DataFrame([media_row])], ignore_index=True)
    df = df.fillna(100)    

    return df

def inicio_SE(historicoSE):

    # Escreva os valores para a região SE na ordem dos meses (jan, fev, mar, ..., dez)
    values_input = [71658, 78130, 52904,  41024, 26938, 24249, 16822, 16036, 16085, 24508, 24449, 43079]

    historicoSE = historicoSE[historicoSE['Ano'] != 2022]

    # Calcula estatísticas e atualiza porcentagens
    historicoSE = calcular_estatisticas_e_atualizar_porcentagens(historicoSE, 2022, values_input)
    historicoSE['Subsistema'] = "SE"
    return historicoSE

def inicio_N(historicoN):

    # Escreva os valores para a região N na ordem dos meses (jan, fev, mar, ..., dez)
    values_input = [35162, 30798, 32737,  29655, 17111, 8829, 4573, 2739, 1763, 1718, 3841, 10253]

    historicoN = historicoN[historicoN['Ano'] != 2022]


    # Calcula estatísticas e atualiza porcentagens
    historicoN = calcular_estatisticas_e_atualizar_porcentagens(historicoN, 2022, values_input)
    historicoN['Subsistema'] = "N"
    return historicoN

def inicio_S(historicoS):

    # Escreva os valores para a região S na ordem dos meses (jan, fev, mar, ..., dez)
    values_input = [2802, 2751, 6990,  10176, 18906, 24482, 7823, 12663, 9727, 20738, 7820, 8163]

    historicoS = historicoS[historicoS['Ano'] != 2022]

    # Calcula estatísticas e atualiza porcentagens
    historicoS = calcular_estatisticas_e_atualizar_porcentagens(historicoS, 2022, values_input)
    historicoS['Subsistema'] = "S"
    return historicoS

def inicio_NE(historicoNE):

    # Escreva os valores para a região NE na ordem dos meses (jan, fev, mar, ..., dez)
    values_input = [17924, 20879, 17281,  7449, 3678, 2857, 2445, 2056, 2234, 1912, 4270, 9539]

    historicoNE = historicoNE[historicoNE['Ano'] != 2022]

    # Calcula estatísticas e atualiza porcentagens
    historicoNE = calcular_estatisticas_e_atualizar_porcentagens(historicoNE, 2022, values_input)
    historicoNE['Subsistema'] = "NE"
    return historicoNE

# Categorização e inicialização do histórico do subsistema SE
historicoSE = historico[historico['Subsistema'] == 'SECO']
historicoSE = inicio_SE(historicoSE)

# Escreva os valores para a região SE na ordem dos meses (jan, fev, mar, ..., dez)
values_inputSE = [71658, 78130, 52904,  41024, 26938, 24249, 16822, 16036, 16085, 24508, 24449, 43079]

# Escreva o ano que você deseja adicionar
anoSE = 2022

# Calcula estatísticas e atualiza porcentagens
historicoSE = calcular_estatisticas_e_atualizar_porcentagens(historicoSE, anoSE, values_inputSE)
historicoSE['Subsistema'] = "SE"

# Categorização do histórico do subsistema N
historicoN = historico[historico['Subsistema'] == 'N']
historicoN = inicio_N(historicoN)

 # Escreva os valores para a região N na ordem dos meses (jan, fev, mar, ..., dez)
values_input = [35162, 30798, 32737,  29655, 17111, 8829, 4573, 2739, 1763, 1718, 3841, 10253]

# Escreva o ano que você deseja adicionar
ano = 2022

# Calcula estatísticas e atualiza porcentagens
historicoN = calcular_estatisticas_e_atualizar_porcentagens(historicoN, ano, values_input)
historicoN['Subsistema'] = "N"


# Categorização do histórico do subsistema S
historicoS = historico[historico['Subsistema'] == 'S']
historicoS = inicio_S(historicoS)

# Escreva os valores para a região S na ordem dos meses (jan, fev, mar, ..., dez)
values_inputS = [2802, 2751, 6990,  10176, 18906, 24482, 7823, 12663, 9727, 20738, 7820, 8163]

# Escreva o ano que você deseja adicionar
anoS = 2022
# Calcula estatísticas e atualiza porcentagens

historicoS = calcular_estatisticas_e_atualizar_porcentagens(historicoS, anoS, values_inputS)
historicoS['Subsistema'] = "S"

# Categorização do histórico do subsistema NE
historicoNE = historico[historico['Subsistema'] == 'NE']
historicoNE = inicio_NE(historicoNE)

 # Escreva os valores para a região NE na ordem dos meses (jan, fev, mar, ..., dez)
values_inputNE = [17924, 20879, 17281,  7449, 3678, 2857, 2445, 2056, 2234, 1912, 4270, 9539]

# Escreva o ano que você deseja adicionar
anoNE = 2022

# Calcula estatísticas e atualiza porcentagens
historicoNE = calcular_estatisticas_e_atualizar_porcentagens(historicoNE, anoNE, values_inputNE)
historicoNE['Subsistema'] = "NE"

df_mlt = pd.concat([historicoSE, historicoS, historicoN, historicoNE]).reset_index(drop=True)

,Ano,(jan)MWmed,(fev)MWmed,(mar)MWmed,(abr)MWmed,(mai)MWmed,(jun)MWmed,(jul)MWmed,(ago)MWmed,(set)MWmed,...,(jul)% med.,(ago)% med.,(set)% med.,(out)% med.,(nov)% med.,(dez)% med.,(min)% med.,(jnd)% med.,(max)% med.,Subsistema
0,1931,69359.690000,103272.430000,107218.270000,80246.410000,55019.580000,40765.810000,31993.5700,25434.170000,25948.880000,...,125.101976,123.944948,132.210640,117.091602,101.535843,99.071047,60.844401,128.873962,256.490831,SE
1,1932,69378.190000,76951.880000,68566.180000,51823.270000,40237.380000,35224.440000,27384.8500,21334.670000,17898.940000,...,107.080856,103.967401,91.195856,107.101383,100.543753,121.880783,42.818393,104.518598,184.086646,SE
2,1933,79952.560000,68553.650000,58429.420000,53341.080000,38425.430000,28005.790000,21773.8100,17187.930000,16172.280000,...,85.140441,83.759646,82.398450,88.931164,72.262547,92.487523,38.687824,93.700592,191.264964,SE
3,1934,57587.480000,52748.750000,52134.030000,43281.560000,30991.250000,22912.100000,18645.7600,13764.920000,13885.410000,...,72.909070,67.078748,70.746751,64.036409,58.436174,96.887221,32.928863,76.953285,137.762534,SE
4,1935,72107.460000,91347.300000,80328.830000,71662.200000,44950.720000,33590.260000,25174.4100,23909.130000,21174.440000,...,98.437544,116.513174,107.884666,161.025812,97.634865,82.118237,50.654144,114.122940,218.523810,SE
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
88,2019,42709.680000,46999.750000,67329.230000,53451.670000,39125.890000,31514.810000,22192.4000,16496.930000,13908.010000,...,86.777221,80.392288,70.861899,59.483613,62.890842,82.760972,33.271168,81.201745,161.067047,SE
89,2020,49519.120000,74708.310000,71300.050000,48035.220000,32061.550000,25213.720000,20618.3100,17616.600000,13233.060000,...,80.622179,85.848627,67.423000,52.389482,60.785195,63.746986,29.643685,82.617308,178.719509,SE
90,2021,46183.750000,51639.170000,52374.430000,35139.060000,25513.160000,21784.740000,15867.8000,12375.880000,11144.700000,...,62.046628,60.309725,56.782717,93.851061,94.151407,89.420033,26.660693,73.116853,125.291716,SE
91,2022,71658.000000,78130.000000,52904.000000,41024.000000,26938.000000,24249.000000,16822.0000,16036.000000,16085.000000,...,65.777762,78.146100,81.953755,103.615050,77.822208,89.644128,38.361811,86.894190,186.904980,SE


In [10]:
df_mlt.to_csv("Mlt_atualizado.csv")

# EARM

In [12]:
import pandas as pd

failure = False; i = 2000
df_earm = pd.DataFrame
while failure == False:
    url = f'https://ons-aws-prod-opendata.s3.amazonaws.com/dataset/ear_subsistema_di/EAR_DIARIO_SUBSISTEMA_{i}.csv'
    try:
        # Lendo o CSV diretamente da URL com delimitador ';'
        dados_earm = pd.read_csv(url, delimiter=';')
    except Exception as e:
        # print(f"Erro ao carregar o arquivo CSV: {e}")
        failure = True
    if i == 2000:
        df_earm = dados_earm
    elif failure == False: 
        df_earm = pd.concat([df_earm, dados_earm])
    i = i + 1
df_earm.drop(columns= 'nom_subsistema', inplace=True)
sum_ear_verif_subsistema_mwmes = df_earm['ear_verif_subsistema_mwmes'].sum()
sum_ear_max_subsistema = df_earm['ear_max_subsistema'].sum()

# Step 2: Add a new column with the ratio
df_earm['verif_max_ratio'] = (sum_ear_verif_subsistema_mwmes / sum_ear_max_subsistema)*100

# If you want to add the total ratio as a new row (optional):
# df_earm.loc['Total'] = df_earm[['ear_verif_subsistema_mwmes', 'ear_max_subsistema']].sum()
# df_earm['verif_max_ratio'] = sum_ear_verif_subsistema_mwmes / sum_ear_max_subsistema

df_earm.reset_index(drop=True, inplace=True)
df_earm

,id_subsistema,ear_data,ear_max_subsistema,ear_verif_subsistema_mwmes,ear_verif_subsistema_percentual,verif_max_ratio
0,NE,2000-01-01,49967.000000,11171.000000,22.360001,54.185491
1,N,2000-01-01,12311.000000,7656.000000,62.189999,54.185491
2,SE,2000-01-01,155759.000000,27853.000000,17.879999,54.185491
3,S,2000-01-01,13976.000000,7094.000000,50.759998,54.185491
4,NE,2000-01-02,49967.000000,11395.000000,22.809999,54.185491
...,...,...,...,...,...,...
36315,S,2024-11-08,20459.242188,12999.841797,63.540199,54.185491
36316,NE,2024-11-09,51691.226562,22658.050781,43.833500,54.185491
36317,N,2024-11-09,15302.396484,9035.113281,59.043800,54.185491
36318,SE,2024-11-09,204615.328125,84558.710938,41.325699,54.185491


In [13]:
df_earm.to_csv('EARM_atualizado.csv')

# PLD

In [9]:
pldH = pd.read_csv('PLD Horário Comercial Historico.csv')
pldH

,Unnamed: 0,Hora,Submercado,Data,Valor
0,0,0,N,2001-06-30,684
1,1,1,N,2001-06-30,684
2,2,2,N,2001-06-30,684
3,3,3,N,2001-06-30,684
4,4,4,N,2001-06-30,684
...,...,...,...,...,...
815731,815731,19,SECO,2024-10-04,643
815732,815732,20,SECO,2024-10-04,"639,75"
815733,815733,21,SECO,2024-10-04,"634,11"
815734,815734,22,SECO,2024-10-04,"624,42"


In [15]:
import pandas as pd

pld = pd.read_csv('modelo_pld.csv', delimiter = ";") #arquivo a ser adicionado

# Primeiro, convertemos o DataFrame de "longo" para "largo"
pld = pld.melt(id_vars=['Hora', 'Submercado'], var_name='Data', value_name='Valor')

# Ajustar o formato da coluna 'Data' para datetime se necessário
pld['Data'] = pd.to_datetime(pld['Data'], format='%d/%m/%Y')

# Limpando o Submercado
submercados = {
    'NORTE': 'N',
    'SUDESTE': 'SECO',
    'SUL': 'S',
    'NORDESTE': 'NE'
}
pld['Submercado'] = pld['Submercado'].replace(submercados)

# Formatando a Data
pld['Data'] = pd.to_datetime(pld['Data'], format='%d/%m/%Y').dt.strftime('%Y-%m-%d')

# Selecionando as colunas na ordem desejada
pldH = pldH[['Hora', 'Submercado', 'Data', 'Valor']]
# Visualizar o DataFrame transposto
pld = pd.concat([pldH, pld]).drop_duplicates(subset=['Hora', 'Submercado', 'Data'], keep='first')
pld


,Hora,Submercado,Data,Valor
0,0,N,2001-06-30,684
1,1,N,2001-06-30,684
2,2,N,2001-06-30,684
3,3,N,2001-06-30,684
4,4,N,2001-06-30,684
...,...,...,...,...
815731,19,SECO,2024-10-04,643
815732,20,SECO,2024-10-04,"639,75"
815733,21,SECO,2024-10-04,"634,11"
815734,22,SECO,2024-10-04,"624,42"


: 

In [14]:
def update_value(df, hora, submercado, data, new_value):
    condition = (df['Hora'] == hora) & (df['Submercado'] == submercado) & (df['Data'] == data)
    if df[condition].empty:
        print("No matching row found.")
    else:
        df.loc[condition, 'Valor'] = new_value
        print(f"Updated 'Valor' for Hora={hora}, Submercado={submercado}, Data={data} to {new_value}.")

#update_value(pld, 1, 'N', '2001-06-30', 700) 
#formato: (pld, hora, 'submercado', 'YYYY-MM-DD', valor correto)



acaba aqui

In [3]:
import requests
import json
dia = '2025-09-14'
submercado = 'NORDESTE'
url = f'https://apise.way2.com.br/v1/pldhorario?apikey=40ffe31b33184ca8ab0a8b551fc33561&dia={dia}&submercado={submercado}'


def obter_dados_pld_semanal(url):
    try:
        # Fazendo a requisição GET para a API
        resposta = requests.get(url)
        resposta.raise_for_status()  # Lança um erro se o status não for 200

        # Obtendo o conteúdo da resposta em formato JSON
        dados = resposta.json()
        
        # Exibindo os dados obtidos
        print("Dados obtidos da API:")
        print(json.dumps(dados, indent=4))
        
        return dados
    except requests.exceptions.HTTPError as errh:
        print(f"Erro HTTP: {errh}")
    except requests.exceptions.ConnectionError as errc:
        print(f"Erro de Conexão: {errc}")
    except requests.exceptions.Timeout as errt:
        print(f"Tempo de Conexão Esgotado: {errt}")
    except requests.exceptions.RequestException as err:
        print(f"Erro ao fazer a requisição: {err}")

# Obtendo os dados da API
dados = obter_dados_pld_semanal(url)


Dados obtidos da API:
[]


In [4]:
dados

[]

In [6]:
def processar_dados(json_data):
    # Extrair a lista de itens
    itens = json_data['body']['items']
    
    # Criar uma lista para armazenar os dados formatados
    dados_formatados = []
    
    # Iterar sobre cada item e extrair os valores
    for item in itens:
        nome = item['name']
        for valor in item['values']:
            dados_formatados.append({
                'Nome': nome,
                'DataHora': valor['dateTime'],
                'Valor': valor['value']
            })
    
    # Converter a lista de dados formatados para um DataFrame do pandas
    df = pd.DataFrame(dados_formatados)
    
    return df

# def processar_dados(json_data):
#     # Verifica se 'body' e 'items' estão presentes
#     if 'body' not in json_data or 'items' not in json_data['body']:
#         raise ValueError("Estrutura do JSON inválida: 'body' ou 'items' não encontrados.")

#     # Extrair a lista de itens
#     itens = json_data['body']['items']
    
#     # Criar uma lista para armazenar os dados formatados
#     dados_formatados = []
    
#     # Iterar sobre cada item e extrair os valores
#     for item in itens:
#         if 'name' in item and 'values' in item:
#             nome = item['name']
#             for valor in item['values']:
#                 if 'dateTime' in valor and 'value' in valor:
#                     dados_formatados.append({
#                         'Nome': nome,
#                         'DataHora': valor['dateTime'],
#                         'Valor': valor['value']
#                     })
    
#     # Converter a lista de dados formatados para um DataFrame do pandas
#     df = pd.DataFrame(dados_formatados)
    
#     return df

# Processar os dados e exibir o DataFrame
df_dados = processar_dados(dados)
print(df_dados)

ValueError: Estrutura do JSON inválida: 'body' ou 'items' não encontrados.

# Tarifas

In [19]:
import pandas as pd
tarifa = pd.read_excel("Histórico Tarifas Distribuidoras (desde 2010).xlsx")
tarifa

,Sigla,Resolução ANEEL,Início Vigência,Fim Vigência,Base Tarifária,Subgrupo,Modalidade,Classe,Subclasse,Detalhe,Acessante,Posto,Unidade,TUSD,TE
0,Amazonas Energia,"REH Nº 1.077, DE 26 DE OUTUBRO DE 2010",2010-11-01,2011-10-31,Base Econômica,A3,Azul,Não se aplica,Não se aplica,APE,Não se aplica,Fora ponta,R$/kW,4.84,0.00
1,Amazonas Energia,"REH Nº 1.077, DE 26 DE OUTUBRO DE 2010",2010-11-01,2011-10-31,Tarifa de Aplicação,A3,Azul,Não se aplica,Não se aplica,APE,Não se aplica,Fora ponta,R$/kW,5.01,0.00
2,Amazonas Energia,"REH Nº 1.077, DE 26 DE OUTUBRO DE 2010",2010-11-01,2011-10-31,Base Econômica,A3,Azul,Não se aplica,Não se aplica,Não se aplica,Não se aplica,Fora ponta,R$/kW,4.84,0.00
3,Amazonas Energia,"REH Nº 1.077, DE 26 DE OUTUBRO DE 2010",2010-11-01,2011-10-31,Tarifa de Aplicação,A3,Azul,Não se aplica,Não se aplica,Não se aplica,Não se aplica,Fora ponta,R$/kW,5.01,0.00
4,Amazonas Energia,"REH Nº 1.077, DE 26 DE OUTUBRO DE 2010",2010-11-01,2011-10-31,Base Econômica,A3,Azul,Não se aplica,Não se aplica,Não se aplica,Não se aplica,Fora ponta,R$/MWh,15.66,0.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
252907,Sulgipe,"REH Nº 3.327, DE 21 DE MAIO DE 2024",2024-05-22,2025-05-21,Tarifa de Aplicação,B4,Convencional,Iluminação pública,Iluminação pública – B4a,SCEE,Não se aplica,Não se aplica,R$/MWh,213.99,27.44
252908,Sulgipe,"REH Nº 3.327, DE 21 DE MAIO DE 2024",2024-05-22,2025-05-21,Base Econômica,B4,Convencional,Iluminação pública,Iluminação pública – B4b,Não se aplica,Não se aplica,Não se aplica,R$/MWh,267.40,132.82
252909,Sulgipe,"REH Nº 3.327, DE 21 DE MAIO DE 2024",2024-05-22,2025-05-21,Tarifa de Aplicação,B4,Convencional,Iluminação pública,Iluminação pública – B4b,Não se aplica,Não se aplica,Não se aplica,R$/MWh,233.45,166.10
252910,Sulgipe,"REH Nº 3.327, DE 21 DE MAIO DE 2024",2024-05-22,2025-05-21,Base Econômica,B4,Convencional,Iluminação pública,Iluminação pública – B4b,SCEE,Não se aplica,Não se aplica,R$/MWh,267.40,19.85


In [57]:
bandeiras = pd.read_excel("2024 09 - HISTÓRICO BANDEIRAS TARIFÁRIAS TODOS SUBMERCADOS (jan2014~).xlsx")
# Calculate the midpoint for rows and columns
mid_row = bandeiras.shape[0] // 2
mid_col = bandeiras.shape[1] // 2

# Split the DataFrame into four parts
df_top_left = bandeiras.iloc[:mid_row, :mid_col]
df_top_right = bandeiras.iloc[:mid_row, mid_col:]
df_bottom_left = bandeiras.iloc[mid_row:, :mid_col]
df_bottom_right = bandeiras.iloc[mid_row:, mid_col:]

# Display the four parts
import pandas as pd

def bottom(df):
    # Exclude the first row and column
    df_processed = df.iloc[1:, 1:]
    
    # Transpose the DataFrame
    df_transposed = df_processed.transpose()
    
    # Set the first row as the header
    df_transposed.columns = df_transposed.iloc[0]
    df_final = df_transposed[1:]  # Exclude the new header row
    
    return df_final.reset_index(drop=True)

def top(df):
    # Excluir a primeira coluna
    df_processed = df.iloc[:13, 1:]
    
    # Transpor o DataFrame
    df_transposed = df_processed.transpose()
    
    # Definir a primeira linha como cabeçalho
    df_transposed.columns = df_transposed.iloc[0]
    df_final = df_transposed[1:]  # Excluir a nova linha de cabeçalho
    
    return df_final.reset_index(drop=True)

bandeiraSeco = top(df_top_left)
bandeiraNordeste = top(df_top_right)
bandeiraSul = bottom(df_bottom_left)
bandeiraNorte = bottom(df_bottom_right)
bandeiraSeco


Unnamed: 1,SE/CO,JAN,FEV,MAR,ABR,MAI,JUN,JUL,AGO,SET,OUT,NOV,DEZ
0,2014,Amarela,Vermelha,Vermelha,Vermelha,Vermelha,Vermelha,Vermelha,Vermelha,Vermelha,Vermelha,Vermelha,Vermelha
1,2015,Vermelha,Vermelha,Vermelha,Vermelha,Vermelha,Vermelha,Vermelha,Vermelha,Vermelha,Vermelha,Vermelha,Vermelha
2,2016,Vermelha,Vermelha,Amarela,Verde,Verde,Verde,Verde,Verde,Verde,Verde,Amarela,Verde
3,2017,Verde,Verde,Amarela,Vermelha 1,Vermelha 1,Verde,Amarela,Vermelha 1,Amarela,Vermelha 2,Vermelha 2,Vermelha 1
4,2018,Verde,Verde,Verde,Verde,Amarela,Vermelha 2,Vermelha 2,Vermelha 2,Vermelha 2,Vermelha 2,Amarela,Verde
5,2019,Verde,Verde,Verde,Verde,Amarela,Verde,Amarela,Vermelha 1,Vermelha 1,Amarela,Vermelha 1,Amarela
6,2020,Amarela,Verde,Verde,Verde,Verde,Verde,Verde,Verde,Verde,Verde,Verde,Vermelha 2
7,2021,Amarela,Amarela,Amarela,Amarela,Vermelha 1,Vermelha 2,Vermelha 2,Vermelha 2,Escassez,Escassez,Escassez,Escassez
8,2022,Escassez,Escassez,Escassez,Escassez,Verde,Verde,Verde,Verde,Verde,Verde,Verde,Verde
9,2023,Verde,Verde,Verde,Verde,Verde,Verde,Verde,Verde,Verde,Verde,Verde,Verde


In [58]:
def substituir_valor(df, ano, mes, novo_valor):
    # Mapeia os meses para suas respectivas colunas
    meses_map = {
        'JAN': 1,
        'FEV': 2,
        'MAR': 3,
        'ABR': 4,
        'MAI': 5,
        'JUN': 6,
        'JUL': 7,
        'AGO': 8,
        'SET': 9,
        'OUT': 10,
        'NOV': 11,
        'DEZ': 12
    }

    # Verifica se o ano já está no DataFrame
    if ano in df.iloc[:, 0].values:
        # Encontra o índice do ano
        idx = df[df.iloc[:, 0] == ano].index[0]

        # Verifica se o mês está mapeado
        if mes in meses_map:
            # Atualiza o valor na célula específica
            coluna = meses_map[mes]
            df.iat[idx, coluna] = novo_valor
        else:
            return "Mês inválido."
    else:
        # Se o ano não está presente, adiciona uma nova linha
        nova_linha = [ano] + [0] * (df.shape[1] - 1)  # Cria nova linha com zeros
        df.loc[len(df)] = nova_linha  # Adiciona a nova linha
        idx = len(df) - 1  # Atualiza o índice da nova linha

        # Verifica se o mês está mapeado
        if mes in meses_map:
            coluna = meses_map[mes]
            df.iat[idx, coluna] = novo_valor
        else:
            return "Mês inválido."
    
    return df

bandeiraSeco = substituir_valor(bandeiraSeco, 2024, 'DEZ', 'TESTE1') #dataframe a ser alterada, yyyy, 'MMM', 'valor adicionado'
bandeiraSeco = substituir_valor(bandeiraSeco, 2025, 'MAI', 'TESTE2')
bandeiraSeco

Unnamed: 1,SE/CO,JAN,FEV,MAR,ABR,MAI,JUN,JUL,AGO,SET,OUT,NOV,DEZ
0,2014,Amarela,Vermelha,Vermelha,Vermelha,Vermelha,Vermelha,Vermelha,Vermelha,Vermelha,Vermelha,Vermelha,Vermelha
1,2015,Vermelha,Vermelha,Vermelha,Vermelha,Vermelha,Vermelha,Vermelha,Vermelha,Vermelha,Vermelha,Vermelha,Vermelha
2,2016,Vermelha,Vermelha,Amarela,Verde,Verde,Verde,Verde,Verde,Verde,Verde,Amarela,Verde
3,2017,Verde,Verde,Amarela,Vermelha 1,Vermelha 1,Verde,Amarela,Vermelha 1,Amarela,Vermelha 2,Vermelha 2,Vermelha 1
4,2018,Verde,Verde,Verde,Verde,Amarela,Vermelha 2,Vermelha 2,Vermelha 2,Vermelha 2,Vermelha 2,Amarela,Verde
5,2019,Verde,Verde,Verde,Verde,Amarela,Verde,Amarela,Vermelha 1,Vermelha 1,Amarela,Vermelha 1,Amarela
6,2020,Amarela,Verde,Verde,Verde,Verde,Verde,Verde,Verde,Verde,Verde,Verde,Vermelha 2
7,2021,Amarela,Amarela,Amarela,Amarela,Vermelha 1,Vermelha 2,Vermelha 2,Vermelha 2,Escassez,Escassez,Escassez,Escassez
8,2022,Escassez,Escassez,Escassez,Escassez,Verde,Verde,Verde,Verde,Verde,Verde,Verde,Verde
9,2023,Verde,Verde,Verde,Verde,Verde,Verde,Verde,Verde,Verde,Verde,Verde,Verde


In [29]:
import requests
import pandas as pd

# Função para obter dados das bandeiras tarifárias
def obter_bandeiras_tarifarias(api_key, data_inicial, data_final):

    agente = 'CELESC-DIS'
    url = f'https://apise.way2.com.br/v1/tarifas?apikey={api_key}&agente={agente}&ano=2023'
    
    try:
        # Fazendo a requisição GET para a API
        resposta = requests.get(url)
        resposta.raise_for_status()  # Lança um erro se o status não for 200

        # Obtendo o conteúdo da resposta em formato JSON
        dados = resposta.json()
        
        # Exibindo os dados obtidos
        print("Dados obtidos da API:")
        print(dados)
        
        return dados
    except requests.exceptions.HTTPError as errh:
        print(f"Erro HTTP: {errh}")
    except requests.exceptions.ConnectionError as errc:
        print(f"Erro de Conexão: {errc}")
    except requests.exceptions.Timeout as errt:
        print(f"Tempo de Conexão Esgotado: {errt}")
    except requests.exceptions.RequestException as err:
        print(f"Erro ao fazer a requisição: {err}")

# Definindo a chave da API e o período de consulta
api_key = '40ffe31b33184ca8ab0a8b551fc33561'
data_inicial = '2020-01-01'
data_final = '2023-06-30'

# Obtendo os dados da API
dados_bandeiras = obter_bandeiras_tarifarias(api_key, data_inicial, data_final)
import pandas as pd
# Verificar se os dados estão presentes e se contêm o campo esperado
try:
    # Extraindo a lista de itens
    itens = dados_bandeiras
    
    # Criar uma lista para armazenar os dados formatados
    dados_formatados = []
    
    # Iterar sobre cada item e extrair os valores
    for item in itens:
        dados_formatados.append({
            'Agente': item.get('agente'),
            'Validade Desde': item.get('validadesde'),
            'Subgrupo': item.get('subgrupo'),
            'Modalidade': item.get('modalidade'),
            'Acessante': item.get('acessante'),
            'Classe': item.get('classe'),
            'Subclasse': item.get('subclasse'),
            'Posto': item.get('posto'),
            'Tarifa Demanda TUSD': item.get('tarifademandatusd'),
            'Tarifa Consumo TUSD': item.get('tarifaconsumotusd'),
            'Tarifa Consumo TE': item.get('tarifaconsumote')
        })
    
    # Converter a lista de dados formatados para um DataFrame do pandas
    df_bandeiras = pd.DataFrame(dados_formatados)
    
    # Exibindo o DataFrame
    print("\nDataFrame de Tarifas Bandeiras:")
    
    # Se necessário, você pode salvar o DataFrame em um arquivo CSV
    # df_bandeiras.to_csv('tarifas_bandeiras.csv', index=False)
except:
    print("Dados não encontrados ou formato inesperado.")

df_bandeiras

Dados obtidos da API:
[{'agente': 'CELESC-DIS', 'validadesde': '2023-08-22T00:00:00.000Z', 'subgrupo': 'A2', 'modalidade': 'AZUL', 'acessante': 'NÃO SE APLICA', 'classe': '', 'subclasse': '', 'posto': 'FP', 'tarifademandatusd': '13.04', 'tarifaconsumotusd': '66.19', 'tarifaconsumote': '276.4'}, {'agente': 'CELESC-DIS', 'validadesde': '2023-08-22T00:00:00.000Z', 'subgrupo': 'A2', 'modalidade': 'AZUL', 'acessante': 'NÃO SE APLICA', 'classe': '', 'subclasse': '', 'posto': 'P', 'tarifademandatusd': '19.52', 'tarifaconsumotusd': '66.19', 'tarifaconsumote': '438.95'}, {'agente': 'CELESC-DIS', 'validadesde': '2023-08-22T00:00:00.000Z', 'subgrupo': 'A2', 'modalidade': 'AZUL APE', 'acessante': 'NÃO SE APLICA', 'classe': '', 'subclasse': '', 'posto': 'FP', 'tarifademandatusd': '13.04', 'tarifaconsumotusd': '8.39', 'tarifaconsumote': '0'}, {'agente': 'CELESC-DIS', 'validadesde': '2023-08-22T00:00:00.000Z', 'subgrupo': 'A2', 'modalidade': 'AZUL APE', 'acessante': 'NÃO SE APLICA', 'classe': '', 'su

,Agente,Validade Desde,Subgrupo,Modalidade,Acessante,Classe,Subclasse,Posto,Tarifa Demanda TUSD,Tarifa Consumo TUSD,Tarifa Consumo TE
0,CELESC-DIS,2023-08-22T00:00:00.000Z,A2,AZUL,NÃO SE APLICA,,,FP,13.04,66.19,276.4
1,CELESC-DIS,2023-08-22T00:00:00.000Z,A2,AZUL,NÃO SE APLICA,,,P,19.52,66.19,438.95
2,CELESC-DIS,2023-08-22T00:00:00.000Z,A2,AZUL APE,NÃO SE APLICA,,,FP,13.04,8.39,0
3,CELESC-DIS,2023-08-22T00:00:00.000Z,A2,AZUL APE,NÃO SE APLICA,,,P,19.52,8.39,0
4,CELESC-DIS,2023-08-22T00:00:00.000Z,A2,GERAÇÃO,CGH FARTURA,,,NA,1.81,0,0
...,...,...,...,...,...,...,...,...,...,...,...
136,CELESC-DIS,2023-08-22T00:00:00.000Z,B3,SCEE - PRÉ-PAGAMENTO,,NA,NA,NA,0,300.39,40.07
137,CELESC-DIS,2023-08-22T00:00:00.000Z,B4,CONVENCIONAL,,ILUMINAÇÃO PÚBLICA,ILUMINAÇÃO PÚBLICA – B4A,NA,0,165.22,160.92
138,CELESC-DIS,2023-08-22T00:00:00.000Z,B4,CONVENCIONAL,,ILUMINAÇÃO PÚBLICA,ILUMINAÇÃO PÚBLICA – B4B,NA,0,180.24,175.54
139,CELESC-DIS,2023-08-22T00:00:00.000Z,B4,SCEE - CONVENCIONAL,,ILUMINAÇÃO PÚBLICA,ILUMINAÇÃO PÚBLICA – B4A,NA,0,165.22,22.04


# Curva Forward

In [18]:
import pandas as pd

df = pd.read_excel("DADOS CURVA FORWARD CONSOLIDADOS.xlsx")

def soma_base(df, dia, subregiao, a_plus):
    # Converter o dia para o formato datetime
    dia = pd.to_datetime(dia)

    # Filtrar o DataFrame pelo dia, mês e ano
    df_filtrado = df[(df['DIA'] == dia)]


    if df_filtrado.empty:
        return f"Não há dados para a data {dia.date()}."

    # Definir a coluna com base na subregião e A+
    if subregiao.lower() == 'norte':
        coluna_base = f'Norte Convencional {a_plus}'
    elif subregiao.lower() == 'sul':
        coluna_base = f'Sul Convencional {a_plus}'
    elif subregiao.lower() == 'nordeste':
        coluna_base = f'Nordeste Convencional {a_plus}'
    elif subregiao.lower() == 'se/co':
        coluna_base = f'SE/CO Convencional {a_plus}'
        return df_filtrado[coluna_base].sum()
    else:
        return "Subregião inválida."

    # Verificar se a coluna existe
    if coluna_base not in df.columns:
        return f"A coluna {coluna_base} não existe no DataFrame."
    
    valor_base = df_filtrado[f'SE/CO Convencional {a_plus}'].iloc[0]

    # Calcular a soma
    soma = df_filtrado[coluna_base].sum()
    
    return (soma+valor_base)

# Selecione apenas as colunas necessárias e renomeie-as
df_organizado = df[['DIA', 'SEMANA', 'MÊS', 
                    'SE/CO CONVENCIONAL', 
                    'Unnamed: 4',  
                    'Unnamed: 5',  
                    'Unnamed: 6', 
                    'SUL CONVENCIONAL', 
                    'Unnamed: 8',  
                    'Unnamed: 9',  
                    'Unnamed: 10',
                    'NORDESTE CONVENCIONAL', 
                    'Unnamed: 12',  
                    'Unnamed: 13',  
                    'Unnamed: 14', 
                    'NORTE CONVENCIONAL', 
                    'Unnamed: 16',  
                    'Unnamed: 17',  
                    'Unnamed: 18',
                    'SE/CO INCENTIVADA',
                    'Unnamed: 20',  
                    'Unnamed: 21',  
                    'Unnamed: 22']]

# Renomear as colunas
df_organizado.columns = ['DIA', 'SEMANA', 'MÊS', 
                         'SE/CO Convencional A+1', 
                         'SE/CO Convencional A+2', 
                         'SE/CO Convencional A+3', 
                         'SE/CO Convencional A+4', 
                         'Sul Convencional A+1', 
                         'Sul Convencional A+2', 
                         'Sul Convencional A+3', 
                         'Sul Convencional A+4',
                         'Nordeste Convencional A+1', 
                         'Nordeste Convencional A+2', 
                         'Nordeste Convencional A+3', 
                         'Nordeste Convencional A+4',
                         'Norte Convencional A+1', 
                         'Norte Convencional A+2', 
                         'Norte Convencional A+3', 
                         'Norte Convencional A+4',
                         'SE/CO Incentivada A+1',
                         'SE/CO Incentivada A+2',
                         'SE/CO Incentivada A+3',
                         'SE/CO Incentivada A+4']

# Exibir o DataFrame organizado
df_organizado = df_organizado.drop(index=0).reset_index(drop=True)
df_organizado
resultado = soma_base(df_organizado, '2012-01-30', 'norte', 'A+3')
resultado

,DIA,SEMANA,MÊS,SE/CO Convencional A+1,SE/CO Convencional A+2,SE/CO Convencional A+3,SE/CO Convencional A+4,Sul Convencional A+1,Sul Convencional A+2,Sul Convencional A+3,...,Nordeste Convencional A+3,Nordeste Convencional A+4,Norte Convencional A+1,Norte Convencional A+2,Norte Convencional A+3,Norte Convencional A+4,SE/CO Incentivada A+1,SE/CO Incentivada A+2,SE/CO Incentivada A+3,SE/CO Incentivada A+4
0,2012-01-02,s01,01/2012,85.5,91.5,93,93,-1.164167,-1.049792,-1.049792,...,-3.9275,-3.720625,-11.271042,-9.399167,-9.188542,-9.004167,113.5,123,124.5,124.5
1,2012-01-09,s02,01/2012,83,89.5,92.5,92.5,-1.115833,-0.917083,-0.917083,...,-3.68625,-3.5425,-11.584583,-9.640833,-9.292083,-9.145833,116.5,123.5,126.5,126.5
2,2012-01-16,s03,01/2012,81.5,89,91.5,91.5,-1.125,-0.99,-0.99125,...,-3.6475,-3.53875,-11.905,-10.21375,-9.92875,-9.79,123.5,129,130.5,130.5
3,2012-01-23,s04,01/2012,81,89,90.5,90,-1.15625,-0.94625,-0.95,...,-3.64875,-3.62625,-10.93625,-9.43,-9.13625,-9.0575,117,125,126.5,126
4,2012-01-30,s05,01/2012,81,88.5,89.5,89.5,-1.09625,-0.91875,-0.92,...,-3.29875,-3.26625,-11.085,-9.42375,-9.23375,-9.1275,112,119,120,120
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
662,2024-09-09,s37,09/2024,166.8,155.65,146.5,143.45,-0.09,-0.1,-0.1,...,-6.94,-6.46,-7.52,-6.86,-6.29,-6.16,200.12,189.08,179.87,176.3
663,2024-09-16,s38,09/2024,165.4,153.77,145.22,142.69,-0.06,-0.07,-0.07,...,-7.15,-6.68,-7.67,-6.91,-6.23,-5.77,198.78,186.88,178.24,175.23
664,2024-09-23,s39,09/2024,170.01,156.8,147.03,143.46,-0.07,-0.08,-0.08,...,-7.47,-6.39,-8.07,-6.99,-6.44,-5.65,203.09,189.83,180.13,176.11
665,2024-09-30,s40,09/2024,170.97,155.66,145.24,142.44,-0.07,-0.07,-0.07,...,-7.91,-7.24,-7.74,-7.41,-6.73,-6.07,203.96,188.24,178.05,175.13
